In [ ]:
import pandas as pd
import os  
import sys
import re
import pandas as pd
import matplotlib.pyplot as plt
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import scipy
from sklearn.model_selection import KFold
from scipy.sparse import csr_matrix, hstack
import re

def get_data():
    train = pd.read_csv(r'C:\Data\train_first.csv')
    test = pd.read_csv(r'C:\Data\predict_first.csv')
    data = pd.concat([train, test])
    print('train %s test %s'%(train.shape,test.shape))
    print('train columns',train.columns)
    return data,train.shape[0],train['Score'],test['Id']

In [23]:
def clear_str(string):
    string = re.sub(r'[0-9a-zA-Z]+', '', string)
    string = string.replace('，','').replace('。','').replace('～','').replace(' ','').replace('！','').\
        replace('<br/>','').replace('；','').replace('）','').replace('（','').replace('.','').\
        replace('“','').replace('”','').replace(',','').replace('【','').replace('】','').replace('~','').\
        replace('\xa0','').replace('《','').replace('》','').replace('<','').replace('>','').replace('/','').strip()
    cut_str = jieba.cut(string.strip())
    list_str = [word for word in cut_str]
    string = ' '.join(list_str)
    return string

In [24]:
def split_discuss(data):
    data['length'] = data['Discuss'].apply(lambda x:len(x))
    data['Discuss'] = data['Discuss'].apply(lambda x: clear_str(x))
    return data

In [25]:
def pre_process():
    data,nrw_train,y,test_id = get_data()
    data = split_discuss(data)
    cv = CountVectorizer(ngram_range=(1,2))
    discuss = cv.fit_transform(data['Discuss'])
    tf = TfidfVectorizer(max_df=10000,ngram_range=(1,2))
    discuss_tf = tf.fit_transform(data['Discuss'])
    # length = csr_matrix(pd.get_dummies(data['length'],sparse=True).values)
    data = hstack((discuss,discuss_tf)).tocsr()
    return data[:nrw_train],data[nrw_train:],y,test_id

In [26]:
def xx_mse_s(y_true,y_pre):
    y_true = y_true
    y_pre = pd.DataFrame({'res':list(y_pre)})

    y_pre['res'] = y_pre['res'].astype(int)
    return 1 / ( 1 + mean_squared_error(y_true,y_pre['res'].values)**0.5)

In [ ]:
X, test, y, test_id = pre_process()
kf = KFold(n_splits=10, shuffle=True, random_state=2018)
kf = kf.split(X)
# get model
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.linear_model import Ridge, LogisticRegression
from mlxtend.classifier import StackingCVClassifier
import lightgbm

cv_pred = []
xx_mse = []
DT = tree.DecisionTreeClassifier()
SVM = SVC()
Ri = Ridge(solver='auto', fit_intercept=True, alpha=0.4, max_iter=250, normalize=False, tol=0.01)
RF = RandomForestClassifier(random_state=10)
EXT = ExtraTreesClassifier()
LR = LogisticRegression()

SCLF = StackingCVClassifier(classifiers=[DT, SVM, Ri, RF, EXT, LR], meta_classifier=lr, use_probas=True, n_folds=3, verbose=3)
for i ,(train_fold,test_fold) in enumerate(kf):
    X_train, X_validate, label_train, label_validate = X[train_fold, :], X[test_fold, :], y[train_fold], y[test_fold]
    SCLF.fit(X_train, label_train)
    val_ = SCLF.predict(X_validate)
    print(xx_mse_s(label_validate, val_))
    cv_pred.append(model_1.predict(test))
    xx_mse.append(xx_mse_s(label_validate, val_))
    
import numpy as np
print('xx_result',np.mean(xx_mse))

s = 0
for i in cv_pred:
    s = s + i

s = s/3
res = pd.DataFrame()
res['Id'] = list(test_id)
res['pre'] = list(s)

res.to_csv(r'C:\Data\t_20180215_1.csv',index=False,header=False)


train (100000, 3) test (30000, 2)
('train columns', Index([u'Id', u'Discuss', u'Score'], dtype='object'))


Building Trie..., from C:\Software\Anaconda2\lib\site-packages\jieba\dict.txt
loading model from cache c:\users\miconron\appdata\local\temp\jieba.cache


dumping model to file cache c:\users\miconron\appdata\local\temp\jieba.cache


loading model cost  2.69099998474 seconds.
Trie has been built succesfully.
